## EHRKit Demo

In this notebook, we demonstrate some of the basic functions that you can utilize from the EHRKit.

### Preparation

In [ ]:
import unittest
import random
import sys, os
import re
import nltk
import json

In [ ]:
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('EHRKit'))))
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(''), '..', 'allennlp')))
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(''), '..', 'summarization', 'pubmed_summarization')))

In [ ]:
from ehrkit import ehrkit

In [ ]:
from getpass import getpass

In [ ]:
try: 
    from config import USERNAME, PASSWORD
except:
    print("Please put your username and password in config.py")
    USERNAME = input('DB_username?')
    PASSWORD = getpass('DB_password?')

In [ ]:
DOC_ID = 1354526 # Temporary!!!

# Number of documents in NOTEEVENTS.
NUM_DOCS = 2083180

# Number of patients in PATIENTS.
NUM_PATIENTS = 46520

# Number of diagnoses in DIAGNOSES_ICD.
NUM_DIAGS = 823933

In [ ]:
def select_ehr(ehrdb, requires_long=False, recursing=False):
    if recursing:
        doc_id = ''
    else:
        doc_id = input("MIMIC Document ID [press Enter for random]: ")
    if doc_id == '':
        # Picks random document
        ehrdb.cur.execute("SELECT ROW_ID FROM mimic.NOTEEVENTS ORDER BY RAND() LIMIT 1")
        doc_id = ehrdb.cur.fetchall()[0][0]
        text = ehrdb.get_document(int(doc_id))
        if len(text.split()) > 200 or not requires_long:
            return doc_id, text
        else:
            return select_ehr(ehrdb, requires_long, True)
    else:
        # Get inputted document
        try:
            text = ehrdb.get_document(int(doc_id))
            return doc_id, text
        except:
            message = 'Error: There is no document with ID \'' + doc_id + '\' in mimic.NOTEEVENTS'
            sys.exit(message)

In [ ]:
def get_nb_dir(ending, SUMM_DIR):
    # Gets path of Naive Bayes model trained on most examples
    dir_nums = []
    for dir in os.listdir(SUMM_DIR):
        if os.path.isdir(os.path.join(SUMM_DIR, dir)) and dir.endswith('_exs_' + ending):
            if os.path.exists(os.path.join(SUMM_DIR, dir, 'nb')):  
                try:
                    dir_nums.append(int(dir.split('_')[0]))
                except:
                    continue
    if len(dir_nums) > 0:
        best_dir_name = str(max(dir_nums)) + '_exs_' + ending
        return best_dir_name
    else:
        return None

In [ ]:
def show_summary(doc_id, text, summary, model_name):
    x = input('Show full EHR (DOC ID %s)? [DEFAULT=Yes]' % doc_id)
    # x = ''
    if x.lower() in ['y', 'yes', '']:
        print('\n\n' + '-'*30 + 'Full EHR' + '-'*30)
        print(text + '\n')
        print('-'*80 + '\n\n')

    print('-'*30 + 'Predicted Summary ' + model_name + '-'*30)
    print(summary)
    print('-'*80 + '\n\n')

In [ ]:
class tests():
    def __init__(self):
        self.ehrdb = ehrkit.start_session(USERNAME, PASSWORD)
        self.ehrdb.get_patients(3)
    

**1.1** Count the total number of patients.

### Test T1

In [ ]:
class tests(tests):
    def test1_1_count_patients(self):
            kit_count = self.ehrdb.count_patients()
            print("Patient count: ", kit_count)

            self.ehrdb.cur.execute("SELECT COUNT(*) FROM mimic.PATIENTS")
            raw = self.ehrdb.cur.fetchall()
            test_count = int(raw[0][0])

In [ ]:
test = tests()
test.test1_1_count_patients()

**1.3** Count the total number of sentences.

In [ ]:
class tests(tests):
    def test1_3_note_info(self):
        self.ehrdb.get_note_events()
        print('output format: SUBJECT_ID, ROW_ID, NoteEvent length')
        lens = [(patient.id, note[0], len(note[1])) for patient in self.ehrdb.patients.values() for note in patient.note_events]
        print(lens)
        

In [ ]:
test = tests()
test.test1_3_note_info()

**1.4** Print the record with the most sentences.

In [ ]:
class tests(tests):
    def test1_4_longest_note(self):
            # Gets longest note among the patient notes queued by get_note_events()
            self.ehrdb.get_note_events()
            pid, rowid, doclen = self.ehrdb.longest_NE()
            print('patient id is:', pid, '\nNoteEvent id is:', rowid, '\nlength: ', doclen)


In [ ]:
test = tests()
test.test1_4_longest_note()

### Test 2

**2.1** Display document given a document ID.

In [ ]:
class tests(tests):
    def test2_1_print_note(self):
            ### There are 2083180 patient records in NOTEEVENTS. ###
            record_id = random.randint(1, NUM_DOCS + 1)
            kit_rec = self.ehrdb.get_document(record_id)
            print("Document with ID %d\n: " % record_id, kit_rec)

            self.ehrdb.cur.execute("select TEXT from mimic.NOTEEVENTS where ROW_ID = %d" % record_id)
            test_rec = self.ehrdb.cur.fetchall()[0][0]


In [ ]:
test = tests()
test.test2_1_print_note()

**2.2** Count the number of documents associated with a given patient given patient ID.

In [ ]:
class tests(tests):
    def test2_2_patient_info(self):
        ### There are records from 46520 unique patients in MIMIC. ###
        patient_id = random.randint(1, NUM_PATIENTS + 1)
        kit_ids = self.ehrdb.get_all_patient_document_ids(patient_id)
        print('Document IDs related to Patient %d: ' % patient_id, kit_ids)
        print("Number of docs related to Patient %d: " % patient_id, len(kit_ids))

        self.ehrdb.cur.execute("select ROW_ID from mimic.NOTEEVENTS where SUBJECT_ID = %d" % patient_id)
        raw = self.ehrdb.cur.fetchall()
        test_ids = ehrkit.flatten(raw)

In [ ]:
test = tests()
test.test2_2_patient_info()

**2.3** List all document IDs.

In [ ]:
class tests(tests):
    def test2_3_doc_ids(self):
            kit_ids = self.ehrdb.list_all_document_ids()

            self.ehrdb.cur.execute("select ROW_ID from mimic.NOTEEVENTS")
            raw = self.ehrdb.cur.fetchall()
            test_ids = ehrkit.flatten(raw)
            print('All document ids: (truncated)')
            print(test_ids[:30])
            print('...')


In [ ]:
test = tests()
test.test2_3_doc_ids()

**2.4** List all patient IDs.

In [ ]:
class tests(tests):
    def test2_4_patient_ids(self):
            kit_ids = self.ehrdb.list_all_patient_ids()

            self.ehrdb.cur.execute("select SUBJECT_ID from mimic.PATIENTS")
            raw = self.ehrdb.cur.fetchall()
            test_ids = ehrkit.flatten(raw)
            print("All patient ids: (truncated)")
            print(test_ids[:30])
            print('...')


In [ ]:
test = tests()
test.test2_4_patient_ids()

**2.5** List all document IDs for a given admission date.

In [ ]:
class tests(tests):
    def test2_5_docs_on_date(self):
        ### Select random date from a date in the database. 
        ### Dates are shifted to future but preserve time, weekday, and seasonality.
        random_id = random.randint(1, NUM_DOCS + 1)
        self.ehrdb.cur.execute("select CHARTDATE from mimic.NOTEEVENTS where ROW_ID = %d" % random_id)
        date = self.ehrdb.cur.fetchall()[0][0]

        kit_ids = self.ehrdb.get_documents_d(date)

        self.ehrdb.cur.execute("select ROW_ID from mimic.NOTEEVENTS where CHARTDATE = \"%s\"" % date)
        raw = self.ehrdb.cur.fetchall()
        test_ids = ehrkit.flatten(raw)
        print(f"Selected date: {date}")
        print(f"Test ids {test_ids[:30]} ...")

In [ ]:
test = tests()
test.test2_5_docs_on_date()

### Test 3 ###

**3.1** Extract all abbreviations from a document, given the document ID.

In [ ]:
class tests(tests):
    def test3_1_extract_abbreviations(self):
        # Defines abbreviation as a string of capitalized letters
        random_id = random.randint(1, NUM_DOCS + 1)
        print("Collecting abbreviations for document %d..." % random_id)
        kit_abbs = self.ehrdb.get_abbreviations(random_id)

        sents = self.ehrdb.get_document_sents(random_id)
        test_abbs = set()
        for sent in sents:
            for word in ehrkit.word_tokenize(sent):
                print(word)
                pattern = r'[A-Z]{2}'  # Only selects words in ALL CAPS
                if re.match(pattern, word):
                    test_abbs.add(word)

        print(kit_abbs)


In [ ]:
test = tests()
test.test3_1_extract_abbreviations()

**3.2** List all document IDs that include keywork "meningitis"

In [ ]:
class tests(tests):
    def test3_2_docs_with_query(self):
        query = "meningitis"
        print('Printing a list of all document ids including query like ', query)
        kit_ids = self.ehrdb.get_documents_q(query)
        print(kit_ids[:30])  # Extremely long list of DOC_IDs
        print("...")

        query = "%"+query+"%"
        self.ehrdb.cur.execute("select ROW_ID from mimic.NOTEEVENTS where TEXT like \'%s\'" % query)
        raw = self.ehrdb.cur.fetchall()
        test_ids = ehrkit.flatten(raw)


In [ ]:
test = tests()
test.test3_2_docs_with_query()

**3.3** List all document IDs that include keywords "Service: SURGERY"

In [ ]:
class tests(tests):
    def test3_3_query_docs(self):
            ### Task 3.3 is the same as task 3.2 with a different query. ###
            query = "Service: SURGERY"
            print('Printing a list of all document ids including query like ', query)
            kit_ids = self.ehrdb.get_documents_q(query)
            print(kit_ids[:30])  # Extremely long list of DOC_IDs
            print("...")

            query = "%"+query+"%"
            self.ehrdb.cur.execute("select ROW_ID from mimic.NOTEEVENTS where TEXT like \'%s\'" % query)
            raw = self.ehrdb.cur.fetchall()
            test_ids = ehrkit.flatten(raw)


In [ ]:
test = tests()
test.test3_3_query_docs()

**3.4** Given a document ID, show a numbered list of all sentences in that document.

In [ ]:
class tests(tests):
    def test3_4_doc_sentences(self):
        doc_id = random.randint(1, NUM_DOCS + 1)
        print('Kit function printing a numbered list of all sentences in document %d' % doc_id)
        # MIMIC EHRs are very messy and sentence tokenizaton often doesn't work
        kit_doc = self.ehrdb.get_document_sents(doc_id)
        ehrkit.numbered_print(kit_doc)

        self.ehrdb.cur.execute("select TEXT from mimic.NOTEEVENTS where ROW_ID = %d " % doc_id)
        raw = self.ehrdb.cur.fetchall()
        test_doc = ehrkit.sent_tokenize(raw[0][0])
        print(test_doc)


In [ ]:
test = tests()
test.test3_4_doc_sentences()

**3.5** Count the number of prescriptions for each unique medication.

In [ ]:
class tests(tests):
    def test3_7_medications(self):
            kit_meds = self.ehrdb.count_all_prescriptions()

            test_meds = {}
            self.ehrdb.cur.execute("select DRUG from mimic.PRESCRIPTIONS")
            raw = self.ehrdb.cur.fetchall()
            meds_list = ehrkit.flatten(raw)
            for med in meds_list:
                if med in test_meds:
                    test_meds[med] += 1
                else:
                    test_meds[med] = 1

            print(meds_list[:30])
            print("...")


In [ ]:
test = tests()
test.test3_7_medications()

### Test T5 ###

**5.4** Count how many patients are labeled as "male" or "female".

In [ ]:
class tests(tests):
    def test5_4_count_gender(self):
        gender = random.choice(['M', 'F'])
        kit_count = self.ehrdb.count_gender(gender)

        self.ehrdb.cur.execute('SELECT COUNT(*) FROM mimic.PATIENTS WHERE GENDER = \'%s\'' % gender)
        raw = self.ehrdb.cur.fetchall()
        test_count = raw[0][0]
        print('Gender:', gender, '\tCount:', str(test_count))


In [ ]:
test = tests()
test.test5_4_count_gender()

### Test T7 ###

**7.1** Creates extractive summary of an EHR with Naive Bayes Algorithm trained on PubMed articles.

In [ ]:
class tests(tests):
    def test7_1_naive_bayes(self):
        from pubmed_naive_bayes import classify_nb
        from get_pubmed_nb_data import build_vecs
        from sklearn.naive_bayes import GaussianNB

        doc_id, text = select_ehr(self.ehrdb)
        body_type = input('Use Naive Bayes model trained from whole body sections or just their body introductions?\n\t'\
                        '[w=whole body, j=just intro, DEFAULT=just intro]: ')
        if body_type == 'w':
            ending = 'body'
        elif body_type in ['j', '']:
            ending = 'intro'
        else:
            sys.exit('Error: Must input \'w\' or \'j.\'')
        SUMM_DIR = os.path.abspath(os.path.join(os.path.dirname('EHRKit'), '..', 'summarization', 'pubmed_summarization'))
        best_dir_name = get_nb_dir(ending, SUMM_DIR)
        if not best_dir_name:
            message = 'No Naive Bayes models of this type have been fit. '\
                        'Would you like to do so now?\n\t[DEFAULT=Yes] '
            response = input(message)
            if response.lower() in ['y', 'yes', '']:
                command = 'python ' + os.path.abspath(os.path.join(os.path.dirname('EHRKit'), '..', 'summarization', 'pubmed_summarization', 'pubmed_naive_bayes.py'))
                os.system(command)
                best_dir_name = get_nb_dir(ending)
            if response.lower() not in ['y', 'yes', ''] or not best_dir_name:
                sys.exit('Exiting.')

        # Fits model to data        
        NB_DIR = os.path.join(SUMM_DIR, best_dir_name, 'nb')
        with open(os.path.join(NB_DIR, 'feature_vecs.json'), 'r') as f:
            data = json.load(f)
        xtrain, ytrain = data['train_features'], data['train_outputs']
        gnb = GaussianNB()
        gnb.fit(xtrain, ytrain)

        # Evaluates on model
        tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        feature_vecs, _ = build_vecs(text, None, tokenizer)
        PCT_SUM = 0.3
        preds = classify_nb(feature_vecs, PCT_SUM, gnb)
        sents = tokenizer.tokenize(text)
        summary = ''
        for i in range(len(preds)):
            if preds[i] == 1:
                summary += sents[i]

        show_summary(doc_id, text, summary, 'Naive Bayes')

In [ ]:
test = tests()
test.test7_1_naive_bayes()

**7.2** Generates abstractive summary of an EHR with pre-trained Distilbart model from Huggingface.

In [ ]:
class tests(tests):
    def test7_2_distilbart_summary(self):
            # Distilbart for summarization. Trained on CNN/ Daily Mail (~4x longer summaries than XSum)
            doc_id, text = select_ehr(self.ehrdb, requires_long=True)
            model_name = 'sshleifer/distilbart-cnn-12-6'
            summary = self.ehrdb.summarize_huggingface(text, model_name)

            show_summary(doc_id, text, summary, model_name)
            print('Number of Words in Full EHR: %d' % len(text.split()))
            print('Number of Words in %s Summary: %d' % (model_name, len(summary.split())))


In [ ]:
test = tests()
test.test7_2_distilbart_summary()

**7.3** Generates abstractive summary of an EHR with pre-trained T5 model from Huggingface.

In [ ]:
class tests(tests):
    def test7_3_t5_summary(self):
            # T5 for summarization. Trained on CNN/ Daily Mail
            doc_id, text = select_ehr(self.ehrdb, requires_long=True)
            model_name = 't5-small'
            summary = self.ehrdb.summarize_huggingface(text, model_name)

            show_summary(doc_id, text, summary, model_name)
            print('Number of Words in Full EHR: %d' % len(text.split()))
            print('Number of Words in %s Summary: %d' % (model_name, len(summary.split())))

In [ ]:
test = tests()
test.test7_3_t5_summary()